In [2]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Создание предсказательной модели и оценивание её качества.

Считаем данные из скачанной открытой гугл-папки

In [4]:
import os
import pandas as pd
path = r'drive/MyDrive/dropbox' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
                else:
                    if df.shape[1] != 1:
                        print(dir_entry_path.split("\\")[-1], df.shape[1])
            except:
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
                else:
                    if df.shape[1] != 1:
                        print(dir_entry_path.split("\\")[-1], df.shape[1])
            except:
                pass

In [5]:
print(len(Target))

1301


In [6]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions

In [7]:
def rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

In [8]:
def integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

Переведём типы движений в числа

In [9]:
classes = {'тояни' : 0, 'месте' : 0,
           'одьб' : 1,'go':1,'стою':1,'покой':1, 'аг' : 1,'stoyat':1,'стойка':1,
           'ег' : 2,'run':2, 'елоси' : 3,
           'естн' : 4, 'одъ' : 4, 'stairs':4,
            'втомо' : 5, 'авто':5, 'ашин' : 5,
           'метро' : 6,
           'лектро' : 7,
           'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [10]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    print(name)
    return -1

In [11]:
answer = [get_action(target) for target in Target]
len(Data), len(answer), len(Target)

drive/MyDrive/dropbox/Игорь Полежаев - Полежаев электричка 1
drive/MyDrive/dropbox/Игорь Полежаев - Полежаев электричка 2
drive/MyDrive/dropbox/Игорь Полежаев - Полежаев электричка 3
drive/MyDrive/dropbox/Игорь Полежаев - Полежаев электричка 4
drive/MyDrive/dropbox/Игорь Полежаев - Полежаев электричка 5
drive/MyDrive/dropbox/Дмитрий Тихановский - sensor Тихановский скейтборд.csv
drive/MyDrive/dropbox/Дмитрий Тихановский - sensor2 Тихановский скейтборд.csv
drive/MyDrive/dropbox/Дмитрий Тихановский - sensor3 Тихановский скейтборд.csv
drive/MyDrive/dropbox/Дмитрий Тихановский - sensor4 Тихановский скейтборд.csv
drive/MyDrive/dropbox/Дмитрий Тихановский - sensor5 Тихановский скейтборд.csv
drive/MyDrive/dropbox/Евгений Измоденов - Измоденов стойка 1.csv
drive/MyDrive/dropbox/Евгений Измоденов - Измоденов стойка 2.csv
drive/MyDrive/dropbox/Евгений Измоденов - Измоденов стойка 3.csv
drive/MyDrive/dropbox/Евгений Измоденов - Измоденов стойка 4.csv


(1301, 1301, 1301)

Обработаем данные и извлечём признаки для создания модели

In [12]:
Data_train = pd.DataFrame(data = {'Ampl':[], 'Moving_x':[], 'Moving_y':[],'Moving_z':[], 'max_freq_a':[],
                                  'max_freq_x':[], 'max_freq_y':[],'max_freq_z':[],
                                  'mean_freq_x':[], 'mean_freq_y':[],'mean_freq_z':[],
                                  'answer':[]})
Data_train

,Ampl,Moving_x,Moving_y,Moving_z,max_freq_a,max_freq_x,max_freq_y,max_freq_z,mean_freq_x,mean_freq_y,mean_freq_z,answer


In [13]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
def max_x_by_y(x, y):
    max_y = 0
    res = x[0]
    for i in range(0, len(y)):
        if y[i] > max_y and x[i] <= 30 and y[i] <= 0.1:
            res = x[i]
            max_y = y[i]
    return res

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1, answer=-1):
    if answer == -1:
        return -1
#     fig, ((ax1, ax2, ax3),(ax4, ax5, ax6),(ax7, ax8, ax9)) = plt.subplots(nrows=3, ncols=3, figsize=(15,10))
    st = 0
    end = len(df)
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
            df['time'] = df['time'].astype(float)
        except :
            return -1
        
#     print(name)
    # обрезаем начало и конец трека
    duration = max(df.time)
    df = df[(df.time > 10) & (df.time < duration - 10)]
    
    if (len(df) ==0):
        return -1
    
    # определим частоту
    freqs = np.array(df.time[1:]) - np.array(df.time[:-1])
    freq = 1/np.mean(freqs)
    
    
#     ax1.plot(df.time, df.iloc[st:end]['gFx'],c='g')
#     ax1.plot(df.time, df.iloc[st:end]['gFy'],c='r')
#     ax1.plot(df.time, df.iloc[st:end]['gFz'],c='b')
#     ax1.set_title('gFx, gFy, gFz по времени')
    
#        show()
########################################################################        
#integration

#вычисляем средний вектор ускорения по треку

    mx = df.iloc[st:end]['gFx'].values.mean() 
    my = df.iloc[st:end]['gFy'].values.mean() 
    mz = df.iloc[st:end]['gFz'].values.mean() 
    
#находим матрицу поворота этого вектора к вектору (0,0,1)

    Vec = [mx,my,mz]
    Point = [0, 0, 1]
    mat = rotation_matrix_from_vectors(Vec, Point)
    
#каждую точку трека поворачиваем на эту матрицу

    new_x, new_y, new_z = rotate(df.iloc[st:end]['gFx'].values, df.iloc[st:end]['gFy'].values, df.iloc[st:end]['gFz'].values, mat)

#проверяем, что новые средние равны примерно (0,0,1). Если это не так, значит, телефон часто поворачивался при движении, и нам не поможет это преобразование

#     print('New means:', mean(new_x), mean(new_y), mean(new_z))

#делим на 250*250 (это нужно было еще при интегрировании)
    new_x = [elem / (freq**2) for elem in new_x]
    new_y = [elem / (freq**2) for elem in new_y]
#из ускорения по z вычитаем 1 (g), чтобы убрать влияние силы тяжести и оставить только ускорение по z
    new_z = [(elem-1)/(freq**2) for elem in new_z]
    
#дважды интегрируем ускорение, чтобы получить перемещение вдоль каждой оси

    int_x = integrate(integrate(new_x))
#     ax7.plot(int_x)
#     ax7.set_title("Перемещение по x")
    moving_x = int_x[-1]
    
    int_y = integrate(integrate(new_y))
#     ax8.plot(int_y)
#     ax8.set_title("Перемещение по y")
    moving_y = int_y[-1]
    
    int_z = integrate(integrate(new_z))
#     ax9.plot(int_z)
#     ax9.set_title("Перемещение по z")
    moving_z = int_z[-1]

########################################################################        
#преобразование Фурье
########################################################################        

    Fs = freq #частота сбора данных
    y = df.iloc[st:end]['gFx'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    
    Min = 0
    
    max_frq_x = max_x_by_y(frq,yabs)
    
    mean_frq_x = np.mean(frq)

    
    y = df.iloc[st:end]['gFy'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    
    max_frq_y = max_x_by_y(frq,yabs)
    mean_frq_y = np.mean(frq)


    y = df.iloc[st:end]['gFz'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
   
    max_frq_z = max_x_by_y(frq,yabs)
    mean_frq_z = np.mean(frq)


    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    y = Acc.values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    

    max_frq_a = max_x_by_y(frq,yabs)
########################################################################        
    
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
    amplitude = Acc.max() - Acc.min()

    Data_train.loc[len(Data_train)] = [amplitude,abs(moving_x),abs(moving_y),abs(moving_z),
                                       max_frq_a,max_frq_x,max_frq_y,max_frq_z,
                                       mean_frq_x,mean_frq_y,mean_frq_z,
                                       int(answer)]
    
    return df

for df, target, ans in zip(Data,Target,answer):
    process_data(df=df, name=target, answer=ans)

In [15]:
Data_train.head(20)

,Ampl,Moving_x,Moving_y,Moving_z,max_freq_a,max_freq_x,max_freq_y,max_freq_z,mean_freq_x,mean_freq_y,mean_freq_z,answer
0,0.062103,0.152437,0.468787,4.974500,4.732512,0.122393,2.896624,0.000000,125.003590,125.003590,125.003590,0.0
1,0.200008,0.081242,0.509455,4.234332,2.811671,2.767739,4.393237,0.000000,125.009549,125.009549,125.009549,0.0
2,0.181625,0.103494,1.133002,2.816271,4.607789,2.912970,4.607789,0.052963,124.992891,124.992891,124.992891,0.0
3,0.069663,2.406644,0.770164,13.714065,4.024148,0.000000,4.024148,0.072290,125.001594,125.001594,125.001594,0.0
4,0.399522,32.343050,2.207324,10.244277,5.985661,0.022587,0.022587,0.000000,124.998667,124.998667,124.998667,0.0
5,1.449805,0.801564,23.829840,0.055366,1.679955,1.679955,0.000000,1.708920,125.011831,125.011831,125.011831,1.0
6,1.738185,1.522133,23.404211,0.843563,3.465310,3.465310,1.732655,0.000000,125.005967,125.005967,125.005967,1.0
7,2.703319,14.396613,0.989242,0.020623,1.685900,1.685900,1.685900,0.000000,125.011465,125.011465,125.011465,1.0
8,1.474270,13.022342,49.106109,0.791074,1.650967,1.688067,1.688067,0.000000,125.000438,125.000438,125.000438,1.0
9,3.496564,2.614811,0.271458,29.609548,1.180885,0.000000,1.180885,1.296658,99.993186,99.993186,99.993186,3.0


Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [16]:
features = 8
classes = 8

In [17]:
!pip install torch

In [18]:
!pip install torchvision

In [19]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as tfs
import torch.utils.data as data_utils

In [20]:
!pip install catboost

In [21]:
import catboost

In [22]:
X = Data_train.drop(columns=['answer'])
y = Data_train['answer']
X_train_origin, X_test_origin, y_train, y_test = train_test_split(X, y, 
                                                       train_size=0.8, 
                                                       random_state=42)
boosting_model = catboost.CatBoostClassifier(n_estimators=200, 
                                             cat_features=[],task_type='GPU')

boosting_model.grid_search({'l2_leaf_reg': np.linspace(0, 5, 4), 'learning_rate': [0.03, 0.1, 0.05],'depth': [4, 6, 8]}, 
                           X_train_origin, 
                           y_train, refit=True)

0:	learn: 1.8215096	test: 1.8298033	best: 1.8298033 (0)
1:	learn: 1.6580849	test: 1.6761590	best: 1.6761590 (1)
2:	learn: 1.5255840	test: 1.5512514	best: 1.5512514 (2)
3:	learn: 1.4115721	test: 1.4436825	best: 1.4436825 (3)
4:	learn: 1.3279609	test: 1.3645241	best: 1.3645241 (4)
5:	learn: 1.2549274	test: 1.2984185	best: 1.2984185 (5)
6:	learn: 1.1913727	test: 1.2382476	best: 1.2382476 (6)
7:	learn: 1.1348701	test: 1.1872658	best: 1.1872658 (7)
8:	learn: 1.0861901	test: 1.1423481	best: 1.1423481 (8)
9:	learn: 1.0406621	test: 1.1037902	best: 1.1037902 (9)
10:	learn: 1.0054537	test: 1.0717628	best: 1.0717628 (10)
11:	learn: 0.9734954	test: 1.0449383	best: 1.0449383 (11)
12:	learn: 0.9419461	test: 1.0173201	best: 1.0173201 (12)
13:	learn: 0.9121638	test: 0.9915801	best: 0.9915801 (13)
14:	learn: 0.8890932	test: 0.9723627	best: 0.9723627 (14)
15:	learn: 0.8609764	test: 0.9502892	best: 0.9502892 (15)
16:	learn: 0.8375755	test: 0.9304215	best: 0.9304215 (16)
17:	learn: 0.8194186	test: 0.91653

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
             

In [33]:
from sklearn.metrics import accuracy_score
y_train = list(map(int,y_train))
y_test = list(map(int,y_test))
y_train_predicted = boosting_model.predict(X_train_origin)
y_test_predicted = boosting_model.predict(X_test_origin)
train_accuracy = accuracy_score(y_train, y_train_predicted)
test_accuracy = accuracy_score(y_test, y_test_predicted)
print('train: ', train_accuracy)
print('test: ', test_accuracy)

train:  0.9455337690631809
test:  0.7304347826086957


# РЕЗУЛЬТАТ

In [35]:
path = r'drive/MyDrive/kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("/")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("/")[-1])
                pass

track_0.csv
track_1.csv
track_10.csv
track_100.csv
track_101.csv
track_102.csv
track_103.csv
track_104.csv
track_105.csv
track_106.csv
track_107.csv
track_108.csv
track_109.csv
track_11.csv
track_110.csv
track_111.csv
track_112.csv
track_113.csv
track_114.csv
track_115.csv
track_116.csv
track_117.csv
track_118.csv
track_119.csv
track_12.csv
track_120.csv
track_121.csv
track_122.csv
track_123.csv
track_124.csv
track_125.csv
track_126.csv
track_127.csv
track_128.csv
track_129.csv
track_13.csv
track_130.csv
track_131.csv
track_132.csv
track_133.csv
track_134.csv
track_135.csv
track_136.csv
track_137.csv
track_138.csv
track_139.csv
track_14.csv
track_140.csv
track_141.csv
track_142.csv
track_143.csv
track_144.csv
track_145.csv
track_15.csv
track_16.csv
track_17.csv
track_18.csv
track_19.csv
track_2.csv
track_20.csv
track_21.csv
track_22.csv
track_23.csv
track_24.csv
track_25.csv
track_26.csv
track_27.csv
track_28.csv
track_29.csv
track_3.csv
track_30.csv
track_31.csv
track_32.csv
track_33.

In [36]:
Data_test = pd.DataFrame(data = {'Ampl':[], 'Moving_x':[], 'Moving_y':[],'Moving_z':[], 'max_freq_a':[],
                                  'max_freq_x':[], 'max_freq_y':[],'max_freq_z':[],
                                  'mean_freq_x':[], 'mean_freq_y':[],'mean_freq_z':[]})
Data_test

,Ampl,Moving_x,Moving_y,Moving_z,max_freq_a,max_freq_x,max_freq_y,max_freq_z,mean_freq_x,mean_freq_y,mean_freq_z


In [37]:
def max_x_by_y(x, y):
    max_y = 0
    res = x[0]
    for i in range(0, len(y)):
        if y[i] > max_y and x[i] <= 30 and y[i] <= 0.1:
            res = x[i]
            max_y = y[i]
    return res

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1, answer=-1):
#     fig, ((ax1, ax2, ax3),(ax4, ax5, ax6),(ax7, ax8, ax9)) = plt.subplots(nrows=3, ncols=3, figsize=(15,10))
    st = 0
    end = len(df)
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except :
            return -1
        try:
            df['time'] = df['time'].astype(float)
            # определим частоту
            freqs = np.array(df.time[1:]) - np.array(df.time[:-1])
            freq = 1/np.mean(freqs)
        except:
            freq = 250
        
#     print(name)
    # обрезаем начало и конец трека
    try:
        duration = max(df.time)
        tmp = df[(df.time > 5) & (df.time < duration - 5)]
        if len(tmp) != 0:
            df = tmp
    except:
        pass
    
    
#     ax1.plot(df.time, df.iloc[st:end]['gFx'],c='g')
#     ax1.plot(df.time, df.iloc[st:end]['gFy'],c='r')
#     ax1.plot(df.time, df.iloc[st:end]['gFz'],c='b')
#     ax1.set_title('gFx, gFy, gFz по времени')
    
#        show()
########################################################################        
#integration

#вычисляем средний вектор ускорения по треку

    mx = df.iloc[st:end]['gFx'].values.mean() 
    my = df.iloc[st:end]['gFy'].values.mean() 
    mz = df.iloc[st:end]['gFz'].values.mean() 
    
#находим матрицу поворота этого вектора к вектору (0,0,1)

    Vec = [mx,my,mz]
    Point = [0, 0, 1]
    mat = rotation_matrix_from_vectors(Vec, Point)
    
#каждую точку трека поворачиваем на эту матрицу

    new_x, new_y, new_z = rotate(df.iloc[st:end]['gFx'].values, df.iloc[st:end]['gFy'].values, df.iloc[st:end]['gFz'].values, mat)

#проверяем, что новые средние равны примерно (0,0,1). Если это не так, значит, телефон часто поворачивался при движении, и нам не поможет это преобразование

#     print('New means:', mean(new_x), mean(new_y), mean(new_z))

#делим на 250*250 (это нужно было еще при интегрировании)
    new_x = [elem / (freq**2) for elem in new_x]
    new_y = [elem / (freq**2) for elem in new_y]
#из ускорения по z вычитаем 1 (g), чтобы убрать влияние силы тяжести и оставить только ускорение по z
    new_z = [(elem-1)/(freq**2) for elem in new_z]
    
#дважды интегрируем ускорение, чтобы получить перемещение вдоль каждой оси

    int_x = integrate(integrate(new_x))
#     ax7.plot(int_x)
#     ax7.set_title("Перемещение по x")
    moving_x = int_x[-1]
    
    int_y = integrate(integrate(new_y))
#     ax8.plot(int_y)
#     ax8.set_title("Перемещение по y")
    moving_y = int_y[-1]
    
    int_z = integrate(integrate(new_z))
#     ax9.plot(int_z)
#     ax9.set_title("Перемещение по z")
    moving_z = int_z[-1]

########################################################################        
#преобразование Фурье
########################################################################        

    Fs = freq #частота сбора данных
    y = df.iloc[st:end]['gFx'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    
    Min = 0
    
#     ax4.plot(frq,yabs) # plotting the spectrum
    max_frq_x = max_x_by_y(frq,yabs)
    mean_frq_x = np.mean(frq)
#     ax4.set_xlim([0,30]) #очень большие частоты нам не нужны
#     ax4.set_ylim([Min,0.1]) #не смотрим на близкие к нулю величины частот
#     ax4.set_title("Частоты по x")

    
    y = df.iloc[st:end]['gFy'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    
#     ax5.plot(frq,yabs) # plotting the spectrum
    max_frq_y = max_x_by_y(frq,yabs)
    mean_frq_y = np.mean(frq)
#     ax5.set_xlim([0,30])
#     ax5.set_ylim([Min,0.1])
#     ax5.set_title("Частоты по y")


    y = df.iloc[st:end]['gFz'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
   
#     ax6.plot(frq,yabs) # plotting the spectrum
    max_frq_z = max_x_by_y(frq,yabs)
    mean_frq_z = np.mean(frq)
#     ax6.set_xlim([0,30])
#     ax6.set_ylim([Min,0.1])
#     ax6.set_title("Частоты по z")


    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    y = Acc.values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    
#     plt.plot(frq,yabs) # plotting the spectrum
#     xlim([0,30])
#     ylim([Min,0.1])
#     plt.title('Частоты общего ускорения')
    max_frq_a = max_x_by_y(frq,yabs)
########################################################################        
    
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
    amplitude = Acc.max() - Acc.min()
#     ax2.plot(Acc)
#     ax2.set_title('Общее ускорение')
#     print(name)
#     plt.show()
    
#     print(name + ' Амплитуда:', Acc.max()-Acc.min())
    track_num = int(name.split('_')[1].split('.')[0])
    Data_test.loc[track_num] = [amplitude,abs(moving_x),abs(moving_y),abs(moving_z),
                                       max_frq_a,max_frq_x,max_frq_y,max_frq_z,
                                       mean_frq_x,mean_frq_y,mean_frq_z]
    
    return df

for df, target, ans in zip(Data,Target,answer):
    process_data(df=df, name=target, answer=ans)

In [38]:
Data_test = Data_test.sort_index(ascending=True)
Data_test.shape

(146, 11)

In [39]:
preds = boosting_model.predict(Data_test)
preds = preds.reshape(1,-1)[0].astype(int64)
len(preds)

146

In [40]:
# Data = torch.tensor(Data_test.values.astype(np.float32)) 
# preds = torch.argmax(boosting_model.predict_proba(X_test_origin)[:, 1], dim=1)
# preds

Создадим таблицу, содержащую номер трека, правильный ответ и предсказание.

In [41]:
Res = pd.DataFrame({'track_num' : np.arange(len(preds)), 'action' : preds})
Res.shape

(146, 2)

In [42]:
Res.to_csv("4_submit.csv", index=False)